# Packages 

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../'
sys.path.append(base_dir)
import os
from utils import *

import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec

from annoy import AnnoyIndex
import polars as pl
import implicit
import scipy.sparse as sps
from utils import str2list
from src.config import raw_data_session_id_dir
from lightgbm import LGBMRanker
from utils import *


/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


# Config 

In [2]:
debug = False

if debug:
    SAMPLE_NUM = 100
else:
    SAMPLE_NUM = None

candidate_path = '../data/candidates/'
# train_data_dir = '.'
# test_data_dir = '.'
# task = 'task1'
# PREDS_PER_SESSION = 100

# num_tree = 100

# model_dir = '../model_training/als_v1'

# # target locales: locales needed for task1
# target_locals = ["DE", 'JP', 'UK']

# submit_file = f'submission_{task}_ALS.parquet'

# Function 

In [3]:
! ls ../data/candidates/

task1_test_nic_top100.parquet  task1_train_nic_top100.parquet
task1_test_w2v_top100.parquet  task1_train_w2v_top100.parquet


In [4]:
# w2v_file = os.path.join(candidate_path, f'task1_{data_type}_w2v_top100.parquet')
# w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))

In [5]:
# length

In [6]:
# list(range(10, 1, -1))

In [7]:
# list(range(100, 0, -1))

In [8]:
# # rank_pct = pl.element().rank(descending=True) #/ pl.col("").count()

# length = w2v_pl.collect().shape[0]
# w2v_pl = w2v_pl.with_columns(
#     pl.Series(values=[list(range(100, 0, -1))]*length, name='w2v_weight')
# )#.head().collect()

In [9]:
# w2v_pl.head().collect()


In [10]:

def get_all_candidates(data_type):
    w2v_file = os.path.join(candidate_path, f'task1_{data_type}_w2v_top100.parquet')
    nic_file = os.path.join(candidate_path, f'task1_{data_type}_nic_top100.parquet')
    w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
    nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))

    cols_to_keep = ['session_id', 'next_item_prediction', 'cg_source']

    w2v_pl = w2v_pl.with_columns(pl.lit('word2vec').alias('cg_source')).select(cols_to_keep).explode('next_item_prediction')
    nic_pl = nic_pl.with_columns(pl.lit('next_item_counter').alias('cg_source')).select(cols_to_keep).explode('next_item_prediction')
    # length = w2v_pl.collect().shape[0]
    # w2v_pl = w2v_pl.with_columns(
    #     pl.Series(values=[list(range(100, 0, -1))]*length, name='w2v_weight')
    # )
    cg_pl = (
            pl.concat([w2v_pl, nic_pl], how='vertical')
                .groupby(['session_id', 'next_item_prediction'])
                .agg(
                    pl.col('cg_source')
                )
                .with_columns(
                    pl.when(pl.col('cg_source').arr.contains(pl.lit('word2vec'))).then(1).otherwise(0).alias('whether_w2v')
                    , pl.when(pl.col('cg_source').arr.contains(pl.lit('next_item_counter'))).then(1).otherwise(0).alias('whether_nic')
                )
                .select(
                    pl.all().exclude('cg_source')
                )
    )
    return cg_pl

# Combine All candidates 

In [11]:
train_candidates = get_all_candidates(data_type='train')

In [12]:
test_candidates = get_all_candidates(data_type='test')

In [13]:
# train_candidates.collect().shape

In [14]:
# test_candidates.collect().shape

In [15]:
# x/train_candidates.select('label').mean().collect()

# Feature Process 

In [16]:
# train_candidates.head(3).collect()

## Load Model 

In [17]:
nic_model = (
    pl.scan_parquet('../model_training/next_item_counter_v2/nic_model.parquet')
        .explode(['next_item_prediction', 'next_item_weight'])
        .select(
            pl.all().exclude('item')
            , pl.col('item').alias('last_prev_item')
        )
            )
print(nic_model.schema)



# w2v_model_file = '../model_training/v2/w2v.model'
# w2vec = Word2Vec.load(w2v_model_file)
# annoy_index = AnnoyIndexer()
# annoy_index.load(annoy_index_file)


{'next_item_prediction': Utf8, 'next_item_weight': Float64, 'last_prev_item': Utf8}


In [18]:
# target_df = train_candidates
# data_type = 'train'

def get_feature(target_df, data_type):
    if data_type == 'train':
        basic_info_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_train.parquet'))
    else:
        basic_info_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task1.parquet'))
    basic_info_pl = basic_info_pl.with_columns(
            pl.col('prev_items').apply(str2list)
        )
    print(f"basci_info_pl:")
    print(basic_info_pl.schema)
    # nic model 

    target_df = (
        target_df.join(basic_info_pl, how='left', on='session_id')
            .with_column(
                pl.col('prev_items').arr.get(-1).alias('last_prev_item')
            )
            .join(nic_model, how='left', on=['last_prev_item', 'next_item_prediction'])
            # .with_columns(
            #     pl.when(pl.col('next_item_weight').is_null()).then(0).otherwise(pl.col('next_item_weight'))
            # )
    )
    if data_type == 'train':
        target_df = (
            target_df
                .with_columns(
                    pl.when(pl.col('next_item')==pl.col('next_item_prediction')).then(1).otherwise(0).alias('target')
                )
        )
    return target_df
# target_df.head(3).collect()

In [19]:
train_candidates = get_feature(target_df=train_candidates, data_type='train')

basci_info_pl:
{'prev_items': Unknown, 'next_item': Utf8, 'locale': Utf8, 'session_id': Int64}


/var/tmp/ipykernel_14981/1310767243.py:17: DeprecationWarning: `LazyFrame.with_column` has been renamed; this redirect is temporary, please use `.with_columns` instead
  target_df.join(basic_info_pl, how='left', on='session_id')


In [20]:
test_candidates = get_feature(target_df=test_candidates, data_type='test')

basci_info_pl:
{'prev_items': Unknown, 'locale': Utf8, 'session_id': Int64}


/var/tmp/ipykernel_14981/1310767243.py:17: DeprecationWarning: `LazyFrame.with_column` has been renamed; this redirect is temporary, please use `.with_columns` instead
  target_df.join(basic_info_pl, how='left', on='session_id')


In [21]:
# train_candidates.head().collect()

In [22]:
# test_candidates.head().collect()

# Ranker 

## Config 

In [23]:
estimator = 10
if debug:
    estimator = 3
    
target = 'target'
feature_cols = [
    'whether_w2v'
    , 'whether_nic'
    # , 'locale'
    , 'next_item_weight'
]

## Train model

In [24]:
train_candidates = train_candidates.sort('session_id')


# .head(100).collect()

In [25]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=estimator, 
    importance_type='gain',
    eval_at=[5]
)

In [26]:
train_candidates = train_candidates.collect()
train_candidates.shape

(69302185, 10)

In [27]:
# train_candidates[feature_cols]

In [28]:
train_candidates.shape

(69302185, 10)

In [29]:
train_candidates.select('target').mean()


target
f64
0.042086


In [30]:
session_lengths_train = train_candidates.groupby('session_id').agg(pl.count()).select('count').to_pandas()['count'].values.tolist()

In [31]:
sum(session_lengths_train)

69302185

In [32]:
ranker = ranker.fit(
    X=train_candidates[feature_cols].to_pandas(),
    y=train_candidates[target].to_pandas(),
    group=session_lengths_train,
    # categorical_feature=['locale'],
    eval_set=[(train_candidates[feature_cols].to_pandas(), train_candidates[target].to_pandas()),
             # (test[feature_cols].to_pandas(), test[target].to_pandas())
             ],
    eval_group=[session_lengths_train,
                # session_lengths_test
               ]
)

/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/sklearn.py:621: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


[1]	valid_0's ndcg@5: 0.455366
[2]	valid_0's ndcg@5: 0.463855
[3]	valid_0's ndcg@5: 0.464695
[4]	valid_0's ndcg@5: 0.464934
[5]	valid_0's ndcg@5: 0.465113
[6]	valid_0's ndcg@5: 0.464936
[7]	valid_0's ndcg@5: 0.4651
[8]	valid_0's ndcg@5: 0.465111
[9]	valid_0's ndcg@5: 0.46512
[10]	valid_0's ndcg@5: 0.465144


In [42]:
import joblib

In [44]:
!mkdir ../model_training/ranker

In [45]:
joblib.dump(
        value=ranker,
        filename='../model_training/ranker/model.pkl'
)

['../model_training/ranker/model.pkl']

### Importance 

In [47]:
impotant_df = pd.DataFrame(
    {
        'features': ranker.feature_name_,
        'importance': ranker.feature_importances_
    }
).sort_values('importance', ascending=False)
impotant_df

,features,importance
2,next_item_weight,7.712072e+06
1,whether_nic,6.595980e+02
0,whether_w2v,3.828390e+02


# Test inference 

In [33]:
test_candidates = test_candidates.collect()
test_candidates.shape

(10220637, 8)

In [34]:
inference = ranker.predict(test_candidates[feature_cols].to_pandas())

In [35]:
test_result = (test_candidates
     .lazy()
     .with_columns(
         pl.Series(name='predict', values=inference)
     )
     .with_columns(
        pl.col('predict').rank(method='ordinal', descending=True).over('session_id').alias('rank')
     )
     .sort(['session_id', 'rank'])
     .filter(pl.col('rank')<=100)
     .groupby(['session_id'])
     .agg(
         pl.col('next_item_prediction')
     )
)
test_result.head().collect()

session_id,next_item_prediction
i64,list[str]
3606249,"[""B099NS1XPG"", ""B08496TCCQ"", … ""B07QH1MYPY""]"
3606250,"[""B004ZXMV4Q"", ""B00R9R5ND6"", … ""B00G358U80""]"
3606251,"[""B08C9Q7QVK"", ""B07G7Q5N6G"", … ""B0816B8H9V""]"
3606252,"[""395535086X"", ""3955350843"", … ""B09N9SNV9S""]"
3606253,"[""B09J8V18FL"", ""B09J8T6TTH"", … ""B0BDLY94TZ""]"


In [36]:
original_test = (pl.scan_parquet('../data/raw_data_session_id/sessions_test_task1.parquet')
                    .with_columns(
                        pl.col('prev_items').apply(str2list)
                    )
                )

In [37]:
original_test.schema

{'prev_items': Unknown, 'locale': Utf8, 'session_id': Int64}

In [38]:
predictions = original_test.join(test_result, how='left', on='session_id').collect()[['locale', 'next_item_prediction']].to_pandas()

In [39]:
predictions.shape

(316971, 2)

In [40]:
# original_test.head()

In [41]:
check_predictions(predictions, test_sessions=original_test.collect().to_pandas(), 
                  # check_products=True, product_df=products
                 )
# Its important that the parquet file you submit is saved with pyarrow backend
predictions.to_parquet('../data/sub_files/rank_v3.parque', engine='pyarrow')

In [46]:
# You can submit with aicrowd-cli, or upload manually on the challenge page.
!aicrowd submission create -c task-1-next-product-recommendation -f "../data/sub_files/rank_v3.parque"

rank_v3.parque ━━━━━━━━━━━━━━━━━━━━━━ 100.0% • 95.9/95.9 MB • 2.7 MB/s • 0:00:0000:0100:02
                                                                                 ╭─────────────────────────╮                                                                                 
                                                                                 │ Successfully submitted! │                                                                                 
                                                                                 ╰─────────────────────────╯                                                                                 
                                                                                       Important links                                                                                       
┌──────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────